# Predict low high models
Model selection for predict lo hi


To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
#%pip install binance-connector==1.13.0
%pip install pandas
%pip install scikit-learn
%pip install tensorflow==2.11.0
%pip install keras==2.11.0
%pip install scikeras


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Feature engineering
Read data from bot's history, resample to equal intervals, create X, y

In [2]:
import os
import glob
import pandas as pd
from datetime import timedelta,date

def read_last_data(strategy):
    """ Read last last day from bot history """
    
    def read_last(symbol: str, name: str):
        data_dir=f"./../data/{strategy}/Xy"
        file_path = sorted([f for f in os.listdir(data_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
        return pd.read_csv(f"{data_dir}/{file_path}", parse_dates=True, index_col="datetime")

    data=read_last("BTCUSDT", "data")
    X=read_last("BTCUSDT", "X")
    y=read_last("BTCUSDT", "y")
    return data,X,y

#strategy="SimpleKerasStrategy"
strategy="LSTMStrategy"
bidask,X_bot,y_bot = read_last_data(strategy)
bidask

,symbol,bid,bid_vol,ask,ask_vol
datetime,,,,,
2023-05-04 23:59:18.498368,BTCUSDT,28820.68,3.26437,28820.69,8.69303
2023-05-04 23:59:18.836063,BTCUSDT,28820.68,3.26326,28820.69,8.69303
2023-05-04 23:59:19.181679,BTCUSDT,28820.68,3.25001,28820.69,8.69303
2023-05-04 23:59:19.520999,BTCUSDT,28820.68,3.24959,28820.69,8.69303
2023-05-04 23:59:19.862379,BTCUSDT,28820.68,3.24959,28820.69,8.69192
...,...,...,...,...,...
2023-05-05 00:18:19.622743,BTCUSDT,28829.57,0.29765,28831.82,0.00065
2023-05-05 00:18:19.964075,BTCUSDT,28829.57,1.09765,28831.82,0.00065
2023-05-05 00:18:20.319516,BTCUSDT,28829.57,1.29765,28831.82,0.00065


In [23]:

def get_targets(bidask):
        """ 
        Calculate targets - bid/ask bounds in future prediction window
        """
        predict_window="60s"
        fut_min=bidask[["bid", "ask"]][::-1].rolling(predict_window).min()[::-1].rename(columns={"bid":"bid_min_fut", "ask":"ask_min_fut"})
        fut_max=bidask[["bid", "ask"]][::-1].rolling(predict_window).max()[::-1].rename(columns={"bid":"bid_max_fut", "ask":"ask_max_fut"})
        return pd.concat([fut_min, fut_max], axis = 1)

y_raw = get_targets(bidask)
#pd.concat([X_bot,y_raw], axis=1).dropna()
dups=X_bot[X_bot.index.duplicated()]
dups.sort_index().tail(10)


,time_hour,time_minute,time_second,time_day_of_week,time_diff,spread,bid_diff,bid_vol_diff,ask_diff,ask_vol_diff,l2_bucket_-10,l2_bucket_-9,l2_bucket_-8,l2_bucket_-7,l2_bucket_-6,l2_bucket_-5,l2_bucket_-4,l2_bucket_-3,l2_bucket_-2,l2_bucket_-1,l2_bucket_0,l2_bucket_1,l2_bucket_2,l2_bucket_3,l2_bucket_4,l2_bucket_5,l2_bucket_6,l2_bucket_7,l2_bucket_8,l2_bucket_9
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-05 00:18:19.622743,0,18,19,4,349855000.0,2.25,0.00,-1.01884,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 00:18:19.622743,0,18,19,4,349855000.0,2.25,0.00,-1.01884,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 00:18:19.622743,0,18,19,4,349855000.0,2.25,0.00,-1.01884,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 00:18:19.622743,0,18,19,4,349855000.0,2.25,0.00,-1.01884,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 00:18:19.964075,0,18,19,4,341332000.0,2.25,0.00,0.80000,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 00:18:19.964075,0,18,19,4,341332000.0,2.25,0.00,0.80000,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 00:18:19.964075,0,18,19,4,341332000.0,2.25,0.00,0.80000,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 00:18:20.319516,0,18,20,4,355441000.0,2.25,0.00,0.20000,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 00:18:20.319516,0,18,20,4,355441000.0,2.25,0.00,0.20000,0.0,0.0,0.00712,0.0,0.0,0.0,0.0,0.0,0.0,6.14059,2.89388,22.35728,10.75168,0.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Resample data to use equal time intervals

In [ ]:
def resampled(X: pd.DataFrame, y: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """ Resample to make equal intervals time series """
    time_interval="10s"
    Xy=pd.concat([X,y], axis=1)
    diffmap = [(c, "sum") for c in Xy.columns if c.endswith("diff")]
    timemap = [(c, "last") for c in Xy.columns if c.startswith("time") and not c.endswith("diff")]
    l2map = [(c, "last") for c in Xy.columns if c.startswith("l2_")]
    
    futmap = [(c, "last") for c in Xy.columns if c.endswith("_fut")]
    colmap = dict(diffmap + timemap + l2map + futmap + [("spread", "last")])
    resampled = Xy.resample(time_interval).agg(colmap).dropna()
    return resampled[X.columns], resampled[y.columns]
    
#X,y = resampled(X_bot, y_raw)
X,y=X_bot,y_raw
pd.concat([X,y], axis=1).dropna()

## Create the model


In [ ]:
from keras import Sequential, Input
from keras.layers import Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scikeras.wrappers import KerasRegressor
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.compose import ColumnTransformer, make_column_selector


test_size=0.3
test_index = int(len(X)*(1-test_size))
X_train, y_train, X_test, y_test = X.iloc[:test_index], y.iloc[:test_index], X.iloc[test_index:], y.iloc[test_index:]

   
def create_model(X_train, y_train, window_size):
        
        model = Sequential()
        model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))
        model.add(Dropout(0.2))
        model.add(LSTM(32))         
        model.add(Dropout(0.2))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(y_train.shape[1], activation='linear'))
        #model.add(Dense(y_train.shape[1], activation='softmax'))
        model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])
        return model
    
    
x_scaler, y_scaler = StandardScaler().fit(X_train), StandardScaler().fit(y_train)
X_train = x_scaler.transform(X_train)
y_train = y_scaler.transform(y_train)

window_size = 10
gen = TimeseriesGenerator(X_train, y_train, length=window_size,
                               sampling_rate=1, batch_size=1)

model = create_model(X_train, y_train, window_size)
model.fit(gen,steps_per_epoch=1000, epochs=10)